In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats

from feature_extraction import build_feature_dataset
from correlacao import ordered_corr, plot_corr, remove_highly_correlated

In [ ]:
OUT = "Dados/dataset_raw2.csv"
METRICS = (
    "mean",
    "std",
    "min",
    "max",
    "rms",
    "ptp",
    "harmonics"
)
FS_LOOKUP = {"LF": 2000}  # sua taxa reamostrada
SPEC = {
    "harmonics": {
        "base": 50.0,     # frequência da rede
        "n": 20,          # até a 20ª harmônica
        "bw": 2.0,        # largura da banda (±bw Hz)
        "method": "welch",# ou "stockwell" (exige pip install stockwell)
        "individual": True  # True para salvar features por harmônica (h_1, h_2, ...)
    }
}

# Criacao

In [ ]:
build_feature_dataset(
    base_dir="Dados/raw",
    output_path=OUT,
    metrics=METRICS,
    include_patterns=["voltage", "lf"],
    # fs_lookup=FS_LOOKUP,
    spec_cfg=SPEC,
    file_format="csv",
    flush_every=500,
)

# Visualização

In [ ]:
# df = pd.read_csv("Dados/dataset_raw.csv")
# df.drop(columns=["sensor", "condicao", "banda", "arquivo", "nome"], inplace=True)
# df.to_csv("Dados/dataset.csv", index=False)

In [ ]:
df = pd.read_csv("Dados/dataset.csv")
display(df.head())

# Correlação

In [ ]:
corr, p_values = ordered_corr(df, 'label')

plot_corr(corr, p_values, True, 0.05)

## Filtro: Remover features com p-value > 0.05

In [ ]:
# Remover colunas não desejadas
to_drop = ["mean", "h_2"]
df_filtered = df.drop(to_drop, axis=1)

corr, p_values = ordered_corr(df_filtered, 'label')

plot_corr(corr, p_values, True, 0.05)

## Filtro: Remover multicolinearidade

In [ ]:
# Remover colunas não desejadas
to_drop = ["mean", "h_2"]
# to_drop += ["h_9", "h_4", "h_20", "h_3", "h_19", "h_17", "rms", "std"]

df_filtered = df.drop(to_drop, axis=1)
corr, p_values = ordered_corr(df_filtered, 'label')

# Usando a função
print("=== Seleção de Features por Correlação ===")
selected_cols = remove_highly_correlated(corr, 'label', threshold=0.5)
df_selected = df_filtered[selected_cols]

# Mostrar a nova matriz de correlação
print("\n=== Nova Matriz de Correlação ===")
corr_new = df_selected.corr()
p_values_new = df_selected.corr(method=lambda x, y: stats.pearsonr(x, y)[1])
plot_corr(corr_new, p_values_new, True, 0.05)

In [ ]:
# SAVE NEW DATASET
df_selected.to_csv("Dados/dataset_selected.csv", index=False)